# Visual perception from your local camera


## 1- Introduction

<img src="http://127.0.0.1:8000/static/remoteAcademy/images/filteredImage.png" width="35%" height="35%" style="float:right;padding:1px"/>
In this exercise we are going to implement a color filter to segment an object in an image provided either by your local camera, a local video file or an external camera controlled by a ROS/ICE plugin. By default, this notebook will get images from your local video device, such as webcams. To resolve this exercise, the student needs to have at least the next knowledge:

- Color spaces (RGB, HSV, etc)
- Python programming skills
- Basic understanding of [OpenCV library](http://opencv.org/)

## 2 - Exercise components

### 2.1 Local Camera

The video device that your computer includes by default (this is: the one in /dev/video0) will be the main component of this exercise. Ir provides a Class which abstracts a Camera from a local device, and provides the methods to keep it constantly updated, so that we will be able to get images from it.

This exercise also allows selecting the video source that we want to use. Although it is intended to solve the exercise of the filter through the student's local camera, he/she can also select another video source (video file stored in the local file system or a camera via ROS / ICE) through the configuration file 'color_filter_conf.yml'.


![selectablesource](http://127.0.0.1:8000/static/remoteAcademy/images/selectablesource.png "Selectable Video Source")

### 2.2 Color Filter component

This component has been developed specifically to carry out this exercise. This component connects to the video source to receive images from it.

The student has to modify this component and add code to accomplish the exercise. In particular, it is required to modify the ``execute()`` method.

### 2.3 Printers

This methods allow to print an image in a Jupyter Notebook for debugging purposes. It will receive processed images from Color Filter to debug our algorithm, or any other image you want to see. 

## 3 - Exercise initialization

To start coding, we need to use ``ColorFilter`` class. Given that a ColorFilter class object starts automatically, you will only need to modify its execute() method, and then run your code through its play() method. Once you have coded your solution to the exercise, go to the end of **block 3** and click "Play Code". You will see the message ``Color filter is running``, and then your code will be executed.

To code the execute() method, follow these instructions:

In [ ]:
# In case you want to use CameraServer (ROS/ICE), uncomment next lines.

#import subprocess
#cameraserver = subprocess.Popen(("cameraserver", "cameraserver_conf.cfg"))

Once our video source is serving images, we can start coding to segment any object. With that putpose, we recommend you to use objects with plain colors, in such a way that the filter values are easier to adjust. We need to modify the ``execute()`` method from Color Filter component with the logic that implements the filter. This method will be called iteratively about 10 times per second. To understand how it works, we are going to print a message in each iteration:

In [ ]:
# Implement execute method
def execute(self):
    print "Running execute iteration"
      
cf.setExecute(execute)

Stop printing updating the method with an empty code:

In [3]:
def execute(self):

    input_image = self.camera.getImage()
    if input_image is not None:
        self.set_color_image(input_image)  
        smooth_image = cv2.GaussianBlur(input_image,(5,5),0)
        HSV_smooth_image = cv2.cvtColor(smooth_image, cv2.COLOR_RGB2HSV)
        lower_boundary = np.array([110,155,0], dtype = "uint8")
        upper_boundary = np.array([179,255,255], dtype = "uint8")

        # PARA UN OBJETO AZUL
        lower_boundary2 = np.array([109,0,0], dtype = "uint8")
        upper_boundary2 = np.array([128,255,255], dtype = "uint8")
        # -------------------

        mask = cv2.inRange(HSV_smooth_image,lower_boundary,upper_boundary)
        mask2 = cv2.inRange(HSV_smooth_image,lower_boundary2,upper_boundary2)
        #self.camera.set_filtered_image(mask)
        input_image_copy = input_image

        mask_copy2 = np.copy(mask2)
        im2_2, contours2, hierarchy2 = cv2.findContours(mask_copy2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        if contours2 != []:
            contour = sorted(contours2, key = cv2.contourArea, reverse = True)[0] 
            # Ordenamos los contornos por su area (de mayor a menor)
            x,y,w,h = cv2.boundingRect(contour)
            rectangle = cv2.rectangle(input_image_copy, (x,y), (x+w,y+h),(255,117,20),2)
            self.set_filtered_image(rectangle)

cf.setExecute(execute)

Code updated


**REMEMBER:** You can use the ``pause()`` method of ColorFilter class to do an "*Academic Pause*", so that you are able to pause your algorithm, make some changes in the ``execute()`` method and setting those changes as shown above, and then resume your algorithm execution by running the ``play()`` method:

1.- Pause
```
cf.pause()
```

2.- Change execute() method
```
def execute(self):
    #make some changes
      
cf.setExecute(execute)
```
3.- Resume
```
cf.play()
```

Or just use the follwing "Play Code/Pause Code" toggle button:

In [1]:
#! /usr/bin/env python

from color_filter import ColorFilter
from color_filter import printImage
import ipywidgets as w
from IPython.display import display
from IPython.display import clear_output

import numpy as np
import cv2
%matplotlib inline

# Init color filter
cf = ColorFilter()

def playcode():
    cf.play()
    
def pausecode():
    cf.pause()
    
playpausebutton = w.ToggleButton(description='Play Code', button_style='success', icon='check', layout=w.Layout(margin='1% 0 0 30%'))

def onclick(change):
    if change['new']:
        playpausebutton.description = "Pause Code"
        playpausebutton.button_style ='danger'
        playpausebutton.icon='stop'
        playcode()
    else: 
        playpausebutton.description = "Play Code"
        playpausebutton.button_style ='success'
        playpausebutton.icon='check'
        pausecode()

playpausebutton.observe(onclick, 'value')

toggle = w.ToggleButton(description='Enable Visualization', layout=w.Layout(margin='1% 0 0 1%'))

def on_click(change):
    if change['new']:
        toggle.description = "Disable Visualization"
        cf.algorithm.visualizationEnabled = True
    else: 
        toggle.description = "Enable Visualization"
        cf.algorithm.visualizationEnabled = False
        clear_output()
        displaybuttons()

toggle.observe(on_click, 'value')

def displaybuttons(): 
    display(w.HBox((playpausebutton, toggle)))
displaybuttons()
cf.algorithm.displaybuttons = displaybuttons

  Chosen source: local camera (index 0)
0 is not a valid device index in this machine.


SystemExit: Please check your camera id (hint: ls /dev)

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 4 - Image manipulation

Color filter receives images from your local camera in principle. To obtain these images you can run this code inside the``execute()`` method:

```
image_input = self.camera.getImage()
```

To debug our code and show the output, there are two images that can be visualized:

- We can visualize an RGB image (three channels), employed to show the images received from the simulator or manipulate them. You can set this images with this code:

```
self.set_color_image(image_three_channels)
```

- We can visualize a gray image (one channels), employed to show the color filter result:

```
self.set_filtered_image(image_one_channels)
```

You can recover these images afterwards with these commands:

In [ ]:
# RGB image
imageRGB = cf.get_color_image()

# Filtered image
filteredGray = cf.get_filtered_image()

To print any of these images in this Notebook, just recover it (get_color_image, get_filtered_image or getImage methd shown above), and then call the ``printImage()`` method, i.e:

In [ ]:
# Example: print the image provided from the camera
imageCamera = cf.camera.getImage()
printImage(imageCamera)

**NOTE:** Each 3 seconds, the ColorFilter Class will automatically try to print a filtered image (it will only print it if you have previously set a filtered image with the ``set_filtered_image(img)`` method as shown above.

You are also allowed to press the 'Show 10 frames' button below to print camera's images every two seconds:

In [ ]:
from color_filter import printVideo

cf.show = False
frames = 10
def showCamera(ev):
    for x in range(0, frames):
        im = cf.camera.getImage()
        cf.algorithm.set_color_image(im)
        # Show color image
        imageCamera = cf.get_color_image()
        #filteredImage = cf.get_filtered_image()
        printVideo(imageCamera)
        #printVideo(filteredImage) 
        clear_output()
    button = w.Button(button_style='info',description="Show " + str(frames) + " frames")
    button.on_click(showCamera)
    display(button)
    
button = w.Button(button_style='info',description="Show " + str(frames) + " frames")
button.on_click(showCamera)
display(button)

## 5 - Programming a color filter segmentation


To accomplish this exercise the student has to implement a color filter that segments a box and detects its position inside the image.

Thus, given an input image like this image:

![Input image](http://127.0.0.1:8000/static/remoteAcademy/images/inputImage.png "Input image")

The expected output would be similar to this image:

![Output image](http://127.0.0.1:8000/static/remoteAcademy/images/filteredImage.png "Output image")

To obtain this result, the proposed pipeline is:

1. Smooth image
2. RGB to HSV conversion
3. Color filter
4. Rectangle approximation
5. Object detection

This steps are detailed in the next sections and can be easily conducted using [OpenCV library](https://opencv.org/ "OpenCV")

### 5.1 - Smooth image

Image smoothing is useful to remove noise or imperfections in image. For this exercise, we recommend to use a *Gaussian Filter*, that can be found in OpenCV library as [GaussianBlur](https://docs.opencv.org/2.4/modules/imgproc/doc/filtering.html?highlight=gaussianblur#gaussianblur). The expected result of this filter from the input image shown in previous section is this:

![Smoothed image](http://127.0.0.1:8000/static/remoteAcademy/images/smoothImage.png "Smoothed image")

### 5.2 - RGB to HSV conversion

The images received from your camera have an RGB color space. This color space is useful to represent digital images but it is also very sensitive to light changes. Therefore, the next step is to convert our RGB image into a HSV image. We recommend to use the [cvtColor](https://docs.opencv.org/2.4/modules/imgproc/doc/miscellaneous_transformations.html#cvtcolor) function from OpenCV.

If we print this image, the expected result will be similar to this image:

![HSV image](http://127.0.0.1:8000/static/remoteAcademy/images/hsvImage.png "HSV image")

### 5.3 - Color filter

Now we can apply our color filter to the HSV image. The OpenCV function [inRange](https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html?highlight=inrange#inrange) can help us to make this color filter.

This function receives two arrays, the first one sets the lower HSV values of the filter, and the second one the upper values. For the first parameter (H), the expected values are in range [0, 180], whereas for S and V the values are in range [0, 255]. Thus, a filter where all pixels would be validated would have this appearance:

```
lower_values = np.array([0,0,0], dtype=np.uint8)
upper_values = np.array([180,255,255], dtype=np.uint8)
```

Note: To represent arrays we employ the [numpy library](http://www.numpy.org/)

Once the maximum and minimum values for each HSV parameters have been properly set, the thresholded image (with one channel) has to be similar to this one:

![Thresholded image](http://127.0.0.1:8000/static/remoteAcademy/images/thresholdImage.png "Thresholded image")

### 5.4 - Rectangle approximation

Trying to find a white object inside a black background is easier than trying to find a colored one inside a changing background. Therefore, from now on, we will use the thresholded image calculated in the previous step.

One option to detect the box could be detecting the object contour with [findContours](https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=findcontours#findcontours) function. This function modifies the input image, so we recommend to create a copy before using it with the [numpy copy](https://docs.scipy.org/doc/numpy/reference/generated/numpy.copy.html) function.

FindContours returns a list of points that defines the object contour. This points can be approximated to polygons using one of the next OpenCV functions: [approxPolyDP](https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=findcontours#approxpolydp) or [boundingRect](https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=findcontours#boundingrect). You can also draw the obtained rectangle with [rectangle](https://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html?highlight=rectangle#rectangle) function.

### 5.5 - Object detection

Since color filtering parameters are not easy to adjust (even in simulated environments), there may be several image regions with rectangles calculated in previous step. One rectangle will belong to the box we are trying to detect, where as the rest will be noisy regions.

In this case, we recommend to filter the calculated rectangles to show only the one belonging to the object you want to segment. You can pick up the proper rectangle setting up some restrictions, like rectangle size or shape deppending on the size of your selected object.

The final result will be the output image we show at the beginning of this section.

![Output image](http://127.0.0.1:8000/static/remoteAcademy/images/filteredImage.png "Output image")

## 6 - Algorithm skeleton

We provide an skeleton where you can code your color filtering following the previous steps:

In [ ]:
def execute(self):
      
    # Get image
    input_image = self.camera.getImage()
    if input_image is None:
        print "Can't get images from camera, is your local camera working?"
        return
    
    if input_image.any(): 
        output_img = np.copy(input_image)
        
        # Smooth image
        # Add your code here
        
        # RGB to HSV conversion
        # Add your code here
        
        # Color filter
        # Add your code here
        
        # Rectangle approximation
        # Add your code here
        
        # Box detection
        # Add your code here

        # Save images
        self.set_color_image(output_img)
        #self.set_filtered_image(thresold_img)

cf.setExecute(execute)

You can see saved images running this code:

In [ ]:
# Show color image
imageCamera = cf.get_color_image()
printImage(imageCamera)

In [ ]:
# Show filtered image
imageCamera = cf.get_filtered_image()
printImage(imageCamera)